In [1]:
!pip install openeo -q

In [2]:
# If running locally, uncomment:
# !pip install --quiet openeo geopandas shapely pandas matplotlib scikit-learn statsmodels

import json, os, datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import shape
import openeo
from openeo.processes import array_interpolate_linear

print('openeo', openeo.__version__)

openeo 0.44.0


## 1) Connect & authenticate to CDSE openEO back‑end
Pick the endpoint. If you want to avoid using partner resources, use the **non‑federated** endpoint.

In [3]:
conn = openeo.connect('openeo.dataspace.copernicus.eu').authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=YTPH-SCCA 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


## 2) Discover collections & processes
We’ll use **SENTINEL2_L2A** (surface reflectance) and the built‑in **ndvi** process.

In [4]:
collections = conn.list_collection_ids()
'SENTINEL2_L2A' in collections, len(collections)

(True, 20)

In [5]:
conn.describe_collection('SENTINEL2_L2A')

{'assets': {'thumbnail': {'href': 'https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2022/07/tallinn_estonia/24383281-1-eng-GB/Tallinn_Estonia_pillars.jpg',
   'roles': ['thumbnail'],
   'title': 'Sentinel 2 L2A',
   'type': 'image/png'}},
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B01',
    'B02',
    'B03',
    'B04',
    'B05',
    'B06',
    'B07',
    'B08',
    'B8A',
    'B09',
    'B11',
    'B12',
    'WVP',
    'AOT',
    'SCL',
    'sunAzimuthAngles',
    'sunZenithAngles',
    'viewAzimuthMean',
    'viewZenithMean']},
  't': {'extent': ['2015-07-04T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-56, 83],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'}},
 'description': 'SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. The SENTINEL-2 Multispectral Instrument (MSI) samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution.\n The level 2A data is atmospherically corrected using Sen2Cor.',
 'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
  'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}},
 'id': 'SENTINEL2_L2A',
 'keywords': ['COPERNICUS',
  'ESA',
  'Orthoimagery',
  'Sentinel-2',
  'MSI',
  'Level-2A',
  'Radiometry',
  'Plant Resource',
  'TOC'],
 'license': 'proprietary',
 'links': [{'href': 'https://sentinels.copernicus.eu/documents/247904/690755/Sentinel_Data_Legal_Notice',
   'rel': 'license',
   'type': 'application/pdf'},
  {'href': 'https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel2.html#sentinel-2-level-2a-top-of-canopy-toc',
   'rel': 'alternate',
   'type': 'application/html'},
  {'href': 'https://dataspace.copernicus.eu/browser/?zoom=11&lat=45.36638&lng=12.49832&themeId=DEFAULT-THEME&visualizationUrl=https%3A%2F%2Fsh.dataspace.copernicus.eu%2Fogc%2Fwms%2F28b654e7-8912-4e59-9e58-85b58d768b3a&datasetId=S2

## 3) Define AOI & time window
- Replace `FIELDS_GEOJSON` with your field polygons (EPSG:4326).
- Adjust dates to your season (sugarcane can span 9–14 months).

In [6]:
# Example: a single polygon (replace with your sugarcane block(s))
FIELDS_GEOJSON = {
  'type':'FeatureCollection',
  'features':[
     {'type':'Feature','properties':{'id':'field_1'}, 'geometry': {
        'type':'Polygon',
        'coordinates': [[[6.8317, 51.9796],[6.8425, 51.9796],[6.8425, 51.9728],[6.8317, 51.9728],[6.8317, 51.9796]]]
     }}
  ]
}

AOI = FIELDS_GEOJSON  # we’ll use these polygons for zonal stats
START = '2024-01-01'
END   = '2025-01-01'

## 4) Build the Sentinel‑2 NDVI data cube
Steps:
1. Load S2 L2A (bands B04, B08, SCL).
2. Mask clouds/snow using the Scene Classification Layer (SCL classes 8–11).
3. Compute NDVI.
4. Resample to a **regular 10‑day series** and linearly interpolate gaps.

In [20]:

# --- A2. Load Sentinel-2 L2A: 10 m bands + SCL for masking
cube = conn.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=AOI,
    temporal_extent=[START, END],
    bands=["B02","B03","B04","B08","SCL"]  # 10 m + SCL
)

# --- A3. Cloud/snow mask using SCL (8,9,10,11: cloud/cirrus/snow).  [5](https://www.researchgate.net/publication/358866180_An_Implementation_of_the_HDBSCAN_Clustering_Algorithm/fulltext/622ae21e97401151d20eba47/An-Implementation-of-the-HDBSCAN-Clustering-Algorithm.pdf)
# Keep only where SCL is not in [8,9,10,11]
def mask_scl(dc):
    scl = dc.band("SCL")
    clear = (scl != 8) & (scl != 9) & (scl != 10) & (scl != 11)
    return dc.mask(clear)

cube = mask_scl(cube)

# --- A4. Derive NDVI and keep 10 m bands
ndvi = cube.ndvi(nir="B08", red="B04")  # built-in process  [7](https://docs.openeo.cloud/federation/backends/processes.html)
cube = cube.filter_bands(["B02","B03","B04","B08"]).merge_cubes(ndvi)

# --- A5. Regularize in time: 10-day median + linear interpolation for missing timestamps.  [6](https://processes.openeo.org/)
cube10 = cube.aggregate_temporal_period(period="dekad", reducer="median")


# --- A6. Export to NetCDF and download
cube10.execute_batch(title='Sugarcane data processing', outputfile='sugarcanedata.nc')
print("Downloaded NetCDF to sugarcanedata.nc")


0:00:00 Job 'j-2508290225044fccad5558f973887104': send 'start'
0:00:13 Job 'j-2508290225044fccad5558f973887104': created (progress 0%)
0:00:19 Job 'j-2508290225044fccad5558f973887104': created (progress 0%)
0:00:25 Job 'j-2508290225044fccad5558f973887104': created (progress 0%)
0:00:33 Job 'j-2508290225044fccad5558f973887104': created (progress 0%)
0:00:43 Job 'j-2508290225044fccad5558f973887104': created (progress 0%)
0:00:55 Job 'j-2508290225044fccad5558f973887104': created (progress 0%)
0:01:11 Job 'j-2508290225044fccad5558f973887104': running (progress N/A)
0:01:30 Job 'j-2508290225044fccad5558f973887104': running (progress N/A)
0:01:55 Job 'j-2508290225044fccad5558f973887104': running (progress N/A)
0:02:25 Job 'j-2508290225044fccad5558f973887104': running (progress N/A)
0:03:02 Job 'j-2508290225044fccad5558f973887104': running (progress N/A)
0:03:49 Job 'j-2508290225044fccad5558f973887104': running (progress N/A)
0:04:47 Job 'j-2508290225044fccad5558f973887104': running (progress

In [ ]:
!pip install rioxarray -q

In [22]:
# --- B0. Environment
# pip install xarray dask[complete] rioxarray rasterio scikit-learn
import os, numpy as np, pandas as pd, xarray as xr
import dask
import dask.array as da
import rioxarray as rxr
from sklearn.cluster import MiniBatchKMeans

In [23]:
ds = xr.open_dataset('sugarcanedata.nc', chunks="auto")

# ---- B1. Normalize to a Dataset with variables: NDVI, B02, B03, B04, B08
def to_band_dataset(ds):
    # Case 1: single data variable with 'bands' dimension
    # (e.g., variable name 'array' with dims ('t','bands','y','x') or ('t','bands','x','y'))
    main = None
    for v in ds.data_vars:
        if "bands" in ds[v].dims:
            main = ds[v]
            break
    if main is not None:
        # Decode band names; they may be bytes/integers
        band_labels = [str(b) for b in main.coords["bands"].values]
        # Some back-ends store original band names like 'B02','B03',...,'NDVI'
        out = {}
        for b in band_labels:
            out[b] = main.sel(bands=b).drop_vars("bands")
        return xr.Dataset(out)

    # Case 2: already separate variables per band
    return ds

bds = to_band_dataset(ds)

# Sanity: ensure NDVI exists; if not, compute from B08/B04
if "NDVI" not in bds:
    bds["NDVI"] = (bds["B08"] - bds["B04"]) / (bds["B08"] + bds["B04"])



In [24]:

# --- B2. Identify dimension names (time/y/x)
time_dim = "t" if "t" in bds.dims else "time"
y_dim = "y" if "y" in bds.dims else ("latitude" if "latitude" in bds.dims else "y")
x_dim = "x" if "x" in bds.dims else ("longitude" if "longitude" in bds.dims else "x")


In [33]:

# --- B3. Per-pixel, whole-series metrics on NDVI
ndvi = bds["NDVI"]

# Integral (AUC) over time:
# If we resampled to exact 10-day steps, AUC ≈ sum(NDVI) * 10 (days).
# More robust: trapezoidal rule with actual time deltas.
t = bds[time_dim]
# compute dt (days) between timesteps, align to mid-intervals
dt = (t.diff(time_dim) / np.timedelta64(1, "D")).astype("float32")
ndvi_mid = 0.5 * (ndvi.isel({time_dim: slice(0, -1)}) + ndvi.isel({time_dim: slice(1, None)}))
AUC = (ndvi_mid * dt.rename({time_dim: ndvi_mid[time_dim].name})).sum(dim=time_dim).rename("NDVI_AUC")

# Peak NDVI and day-of-year (DOY) at peak
# NDVI maximum value (as before)
NDVI_MAX = ndvi.max(dim=time_dim).rename("NDVI_MAX")

# Coordinate of the maximum per pixel (returns datetime64 at each y,x)
peak_time = ndvi.idxmax(dim=time_dim)

# If a pixel is all-NaN over time, idxmax -> NaT; handle that:
DOY_MAX = peak_time.dt.dayofyear.fillna(-1).astype("int16").rename("NDVI_DOY_MAX")

# Mean, Std, CV
NDVI_MEAN = ndvi.mean(dim=time_dim).rename("NDVI_MEAN")
NDVI_STD = ndvi.std(dim=time_dim).rename("NDVI_STD")
NDVI_CV = (NDVI_STD / (NDVI_MEAN + 1e-6)).rename("NDVI_CV")

metrics = xr.merge([AUC, NDVI_MAX, DOY_MAX, NDVI_MEAN, NDVI_STD, NDVI_CV])


In [34]:

# --- B4. Per-pixel derived features from 10 m bands (examples)
# GNDVI (Green NDVI) uses 10 m bands only: (NIR - Green)/(NIR + Green)
GNDVI = ((bds["B08"] - bds["B03"]) / (bds["B08"] + bds["B03"])).mean(dim=time_dim).rename("GNDVI_MEAN")

# RGB reflectance temporal mean (useful for clustering/context)
B02_MEAN = bds["B02"].mean(dim=time_dim).rename("B02_MEAN")
B03_MEAN = bds["B03"].mean(dim=time_dim).rename("B03_MEAN")
B04_MEAN = bds["B04"].mean(dim=time_dim).rename("B04_MEAN")

features = xr.merge([metrics, GNDVI, B02_MEAN, B03_MEAN, B04_MEAN])

# --- B5. Per-pixel unsupervised clustering (MiniBatchKMeans)
# Stack y/x -> pixels; sample to fit centers, then predict for all pixels.
n_clusters = 4
feat_names = ["NDVI_AUC","NDVI_MAX","NDVI_CV","GNDVI_MEAN","B02_MEAN","B03_MEAN","B04_MEAN"]
F = features[feat_names].to_array().transpose(..., y_dim, x_dim)  # dims: variable, y, x
F2d = F.stack(pixel=(y_dim, x_dim)).transpose("pixel","variable")  # pixels x variables
F2d = F2d.astype("float32").fillna(0.0)

# Subsample for fitting (e.g., 200k pixels)
sample = F2d.isel(pixel=slice(0, min(200_000, F2d.sizes["pixel"]))).compute()
km = MiniBatchKMeans(n_clusters=n_clusters, random_state=0, batch_size=2048, n_init="auto")
km.fit(sample.values)

# Predict for all pixels in chunks
labels = []
chunk = 2_000_000
for i in range(0, F2d.sizes["pixel"], chunk):
    block = F2d.isel(pixel=slice(i, min(i+chunk, F2d.sizes["pixel"]))).compute()
    labels.append(km.predict(block.values))
import numpy as np
labels = np.concatenate(labels).astype("int16")

clusters = xr.DataArray(labels, dims=["pixel"], coords={"pixel": F2d["pixel"]})\
           .unstack("pixel").rename("QUALITY_CLUSTER")


In [35]:

# --- B6. Merge and write out results (NetCDF and GeoTIFFs)
out = xr.merge([features, clusters])

# Preserve georeferencing & CRS if present
# If the dataset already has spatial reference attributes, rioxarray can carry them through.
if hasattr(ndvi, "rio"):
    try:
        out = out.rio.write_crs(ndvi.rio.crs or "EPSG:4326", inplace=False)
    except Exception:
        pass

# NetCDF
out.to_netcdf("local_per_pixel_metrics_and_clusters.nc")

# Example GeoTIFF exports (per-pixel rasters)
out["NDVI_AUC"].rio.to_raster("NDVI_AUC.tif")
out["NDVI_MAX"].rio.to_raster("NDVI_MAX.tif")
out["QUALITY_CLUSTER"].rio.to_raster("QUALITY_CLUSTER.tif")